## Extracting citations from DH Conference papers

This notebook presents some basic results obtained extracting citations from the abstracts of all (ADHO) DH conferences from 2015 to 2020 and  from DhQ journal articles. The dataset is available [here](https://github.com/lehkost/ToolXtractor/)

### Preamble

In [1]:
import ast
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bs4 import BeautifulSoup

In [2]:
def read_tei(tei_file):
    with open(tei_file, 'r') as tei:
        soup = BeautifulSoup(tei, 'lxml')
        return soup
    raise RuntimeError('Cannot generate a soup from the input')

In [ ]:
def elem_to_text(elem, default=''):
    if elem:
        return elem.getText(separator=' ', strip=True)
    else:
        return default

In [4]:
from dataclasses import dataclass

@dataclass
class Person:
    firstname: str
    middlename: str
    surname: str

In [5]:
class TEIFile(object):
    def __init__(self, filename):
        self.filename = filename
        self.soup = read_tei(filename)
        self._text = None
        self._title = ''
        self._abstract = ''

    @property
    def doi(self):
        idno_elem = self.soup.find('idno', type='DOI')
        if not idno_elem:
            return ''
        else:
            return idno_elem.getText()

    @property
    def title(self):
        if not self._title:
            if  not self.soup.title:
                self._title = "na"
            else:
                self._title = self.soup.title.getText()
        return self._title

    @property
    def abstract(self):
        if not self._abstract:
            abstract = self.soup.abstract.getText(separator=' ', strip=True)
            self._abstract = abstract
        return self._abstract

    @property
    def authors(self):
        #authors_in_header = self.soup.analytic.find_all('author')
        authors_in_header = self.soup.find_all('author')

        result = []
        for author in authors_in_header:
            persname = author.persname
            if not persname:
                continue
            firstname = elem_to_text(persname.find("forename"))#, type="first"))
            middlename = elem_to_text(persname.find("forename", type="middle"))
            surname = elem_to_text(persname.surname)
            person = Person(firstname, middlename, surname)
            result.append(person)
        return result
    
    @property
    def bibliography(self):
        bibliography = self.soup.find_all('bibl')
        result = []
        for bibl in bibliography:
            if not bibl:
                continue
            
            result.append(elem_to_text(bibl))
        return result
    
    @property
    def text(self):
        if not self._text:
            divs_text = []
            for div in self.soup.body.find_all("div"):
                # div is neither an appendix nor references, just plain text.
                if not div.get("type"):
                    div_text = div.get_text(separator=' ', strip=True)
                    divs_text.append(div_text)

            plain_text = " ".join(divs_text)
            self._text = plain_text
        return self._text

In [19]:
def get_citation_cn (df_dois_values, default=''):
    df_cn_citations = pd.DataFrame (columns = ['doi','cn_citation']);
    import requests;
    headers_dict = {"Accept": "text/x-bibliography", "locale":"en-EN"};
    for vard in df_orig_dois_values:
        if ( vard != "" and vard!=None):
            print(vard)
            try:
                rd =requests.get("http://doi.org/"+vard, headers=headers_dict, timeout=20)
                # print("result: "+r.content.decode("utf-8"))
                if ('DOI Not Found'in rd.text):
                    df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': 'Not Found'}, ignore_index=True)
                else:
                    df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': rd.content.decode("latin-1")}, ignore_index=True)
            except requests.exceptions.ConnectionError:
              #  print(var)
                df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': int(503)}, ignore_index=True)
            except requests.exceptions.ConnectTimeout:
              #  print(var)
                df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': int(408)}, ignore_index=True)
            except requests.exceptions.ReadTimeout:
                df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': int(408)}, ignore_index=True)
        else:
           # print(var ,0)
            df_cn_citations = df_cn_citations.append({'doi': 'none', 'cn_citation': int(400)}, ignore_index=True)
    return df_cn_citations

In [70]:
def get_citation_from_SSHOC (df_dois_values, default=''):
    df_cn_citations = pd.DataFrame (columns = ['doi','cn_citation']);
    import requests;
    #headers_dict = {"Accept": "text/x-bibliography", "locale":"en-EN"};
    for vard in df_dois_values:
        if ( vard != "" and vard!=None):
            print(vard)
            try:
                rd =requests.get("http://v4e-lab.isti.cnr.it/citationservice/citharvester/getmetadatahtml?pid=https://doi.org/"+vard+"&token=test", timeout=75)
                #print("result: "+rd.content.decode("latin-1"))
                if ('DOI Not Found'in rd.text):
                    df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': 'Not Found'}, ignore_index=True)
                else:
                    df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': rd.content.decode("latin-1")}, ignore_index=True)
            except requests.exceptions.ConnectionError:
              #  print(var)
                df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': int(503)}, ignore_index=True)
            except requests.exceptions.ConnectTimeout:
              #  print(var)
                df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': int(408)}, ignore_index=True)
            except requests.exceptions.ReadTimeout:
                df_cn_citations = df_cn_citations.append({'doi': vard, 'cn_citation': int(408)}, ignore_index=True)
        else:
           # print(var ,0)
            df_cn_citations = df_cn_citations.append({'doi': 'none', 'cn_citation': int(400)}, ignore_index=True)
    return df_cn_citations

In [221]:
def get_HTTP_Status(urls):
    import requests;
    df_http_status = pd.DataFrame (columns = ['DOI','status'])
    for var in urls:
        #print(var)
        if ( var != ""):
            try:
                r =requests.get("https://doi.org/"+var,timeout=21)
                print("result: "+var+ " ",r.status_code)
                df_http_status = df_http_status.append({'DOI': var, 'status': int(r.status_code)}, ignore_index=True)
            except requests.exceptions.ConnectionError:
              #  print(var)
                df_http_status = df_http_status.append({'DOI': var, 'status': int(503)}, ignore_index=True)
            except requests.exceptions.ConnectTimeout:
                df_http_status = df_http_status.append({'DOI': var, 'status': int(408)}, ignore_index=True)
            except requests.exceptions.ReadTimeout:
                df_http_status = df_http_status.append({'DOI': var, 'status': int(408)}, ignore_index=True)
            except requests.exceptions.RequestException:
                df_http_status = df_http_status.append({'DOI': var, 'status': int(500)}, ignore_index=True)
            except TypeError:
                df_http_status = df_http_status.append({'DOI': var, 'status': int(400)}, ignore_index=True)
        else:
            df_http_status = df_http_status.append({'DOI': var, 'status': int(400)}, ignore_index=True)
    return df_http_status;

In [6]:
import multiprocessing

In [7]:
from os.path import basename, splitext

def basename_without_ext(path):
    base_name = basename(path)
    stem, ext = splitext(base_name)
    if stem.endswith('.tei'):
        # Return base name without tei file
        return stem[0:-4]
    else:
        return stem

In [8]:
def tei_to_csv_entry(tei_file):
    tei = TEIFile(tei_file)
    print(f"Handled {tei_file}")
    base_name = basename_without_ext(tei_file)
    return base_name, tei.authors, tei.title, tei.bibliography#, tei.abstract

In [ ]:
from multiprocessing.pool import Pool
pool = Pool()

### Import the DH conference abstracts (2016-20020) and Dhq articles

The dataset is downloaded from https://github.com/lehkost/ToolXtractor/

In [9]:
import glob
from pathlib import Path
papers15 = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DH/xml/2015/").glob('*.xml'))
papers16 = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DH/xml/2016/").glob('*.xml'))
papers17 = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DH/xml/2017/").glob('*.xml'))
papers18 = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DH/xml/2018/").glob('*.xml'))
papers19 = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DH/xml/2019/").glob('*.xml'))
papers20 = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DH/xml/2020/").glob('*.xml'))
papersdhq = sorted(Path("/Users/cesare/git/SSHOCCitationService/dataset/ToolXtractor/data/DHQ/").glob('*.xml'))

In [11]:
csv_entries15 = pool.map(tei_to_csv_entry, papers15)
csv_entries16 = pool.map(tei_to_csv_entry, papers16)
csv_entries17 = pool.map(tei_to_csv_entry, papers17)
csv_entries18 = pool.map(tei_to_csv_entry, papers18)
csv_entries19 = pool.map(tei_to_csv_entry, papers19)
csv_entries20 = pool.map(tei_to_csv_entry, papers20)
csv_entriesdhq=pool.map(tei_to_csv_entry, papersdhq)

In [12]:
result_csv0 = pd.DataFrame(csv_entries15, columns=['ID', 'Authors', 'Title', 'Bibliography'])
result_csv1 = result_csv0.append(pd.DataFrame(csv_entries16, columns=['ID', 'Authors', 'Title', 'Bibliography']))
result_csv2 = result_csv1.append(pd.DataFrame(csv_entries17, columns=['ID', 'Authors', 'Title', 'Bibliography']))
result_csv3 = result_csv2.append(pd.DataFrame(csv_entries18, columns=['ID', 'Authors', 'Title', 'Bibliography']))
result_csv4 = result_csv3.append(pd.DataFrame(csv_entries19, columns=['ID', 'Authors', 'Title', 'Bibliography']))
result_csv5 = result_csv4.append(pd.DataFrame(csv_entries20, columns=['ID', 'Authors', 'Title', 'Bibliography']))
result_csv = result_csv5.append(pd.DataFrame(csv_entriesdhq, columns=['ID', 'Authors', 'Title', 'Bibliography']))
result_csv.count()

ID              2788
Authors         2788
Title           2788
Bibliography    2788
dtype: int64

### Select the papers having the TEI \<bibl\>   elements.

The dataset contains 2788 documents structured according TEI document model.

These documents are parsed to individuate those having the \<bibl\> (bibliographic citation) element, this element  contains a loosely-structured bibliographic citation of which the sub-components may or may not be explicitly tagged. There are 1624 documents having this element.

In [13]:
test_csv=result_csv[result_csv['Bibliography'].str.len()>0]
test_csv.count()

ID              1624
Authors         1624
Title           1624
Bibliography    1624
dtype: int64

In [154]:
# all citations
my_df=test_csv[['ID','Title','Bibliography']]
my_exp_df=my_df.explode('Bibliography')
my_exp_df.count()

ID              26028
Title           26028
Bibliography    26028
dtype: int64

The total number of citations individuated in the dataset is 26028.

### Find citations having a DOI
Regular expressions are used to parse all citations in order to individuate those having a DOI: there are 1162 (on 26028) citations with a DOI.

In [208]:
import re
regex = re.compile(r'\b(10[.][0-9]{4,}(?:[.][0-9]+)*/(?:(?!["&\'<>])\S)+)\b', re.IGNORECASE)
#regex = re.compile(r'\b(10.\d{0,}/[-._;()/:A-Z0-9])\b', re.IGNORECASE)
#regex = re.compile(
#        r'/^10.\d{4,9}/[-._;()/:A-Z0-9]+$/i'
#        r'/^10.1002/[^\s]+$/i'
#        r'/^10.\d{4}/\d+-\d+X?(\d+)\d+<[\d\w]+:[\d\w]*>\d+.\d+.\w+;\d$/i'
#        r'/^10.1021/\w\w\d+$/i'
#        r'/^10.1207/[\w\d]+\&\d+_\d+$/i', re.IGNORECASE)
df_refs=my_exp_df.Bibliography.values
df_refs_with_doi = pd.DataFrame(columns=["Reference", "DOI"])
references=[]
DOIs=[]
for reference in df_refs:
    mydoi=re.search(regex, reference)
    if mydoi:
        references.append(reference);
        DOIs.append(mydoi[0]);
df_refs_with_doi['Reference']=references;
df_refs_with_doi['DOI']=DOIs;
df_refs_with_doi.drop_duplicates(inplace=True)
df_refs_with_doi.count()

Reference    1162
DOI          1162
dtype: int64

In [209]:
#Example: five citations that have DOIs
df_refs_with_doi.head()
df_refs_with_doi.head().style.set_properties(subset=['Reference'], **{'width': '600px'})

,Reference,DOI
0,"Byrne, G., and Goddard, L. (2010). The Strongest Link: Libraries and Linked Data. D-Lib Magazine , 16 (11/12), doi:10.1045/november2010-byrne.",10.1045/november2010-byrne
1,"Lampert, C. K., and Southwick, S. B. (2013). Leading to Linking: Introducing Linked Data to Academic Library Digital Collections. Journal of Library Metadata , 13 (2–3): 230–53, doi:10.1080/19386389.2013.826095.",10.1080/19386389.2013.826095
2,"Singer, R. (2009). Linked Library Data Now! Journal of Electronic Resources Librarianship , 21 (2): 114–26, doi:10.1080/19411260903035809.",10.1080/19411260903035809
3,"Thomas, L. and Solomon, D. (2014). Active Users: Project Development and Digital Humanities Pedagogy. CEA Critic, 76 (2) (July): 211–20, DOI:10.1353/cea.2014.0014.",10.1353/cea.2014.0014
4,"Farquhar, A. and Baker, J. (2014). Interoperable Infrastructures for Digital Research: A Proposed Pathway for Enabling Transformation. Digital Humanities 2014, http://dx.doi.org/10.6084/m9.figshare.1092550%20.",10.6084/m9.figshare.1092550%20


### Check DOIs: HTTP return status

All DOIs are invoked and the HTTP return status is captured.

In [224]:
df_urls=df_refs_with_doi.DOI.values
df_http_status = pd.DataFrame (columns = ['DOI','status'])
import requests
for var in df_urls:
    #print(var)
    if ( var != ""):
        try:
            r =requests.get("https://doi.org/"+var,timeout=21)
            print("result: "+var+ " ",r.status_code)
            df_http_status = df_http_status.append({'DOI': var, 'status': int(r.status_code)}, ignore_index=True)
        except requests.exceptions.ConnectionError:
          #  print(var)
            df_http_status = df_http_status.append({'DOI': var, 'status': int(503)}, ignore_index=True)
        except requests.exceptions.ConnectTimeout:
            print(var)
            df_http_status = df_http_status.append({'DOI': var, 'status': int(408)}, ignore_index=True)
        except requests.exceptions.ReadTimeout:
         #   print(var)
            df_http_status = df_http_status.append({'DOI': var, 'status': int(408)}, ignore_index=True)
        except requests.exceptions.RequestException:
         #   print(var)
            df_http_status = df_http_status.append({'DOI': var, 'status': int(500)}, ignore_index=True)
        except TypeError:
        #    print(var)
            df_http_status = df_http_status.append({'DOI': var, 'status': int(400)}, ignore_index=True)
    else:
       # print(var ,0)
        df_http_status = df_http_status.append({'DOI': var, 'status': int(400)}, ignore_index=True)
df_http_status.head()

result: 10.1045/november2010-byrne  200
result: 10.1080/19386389.2013.826095  200
result: 10.1080/19411260903035809  200
result: 10.1353/cea.2014.0014  503
result: 10.6084/m9.figshare.1092550%20  200
result: 10.5772/16327  200
result: 10.1093/llc/fqu026  403
result: 10.1093/llc/fqq026  403
result: 10.1037/h0047157  200
result: 10.1016/j.eswa.2010.09.103  200
result: 10.3115/1073336.1073347  403
result: 10.1007/978-1-4020-6264-3_67  200
result: 10.1093/llc/fqt037  403
result: 10.2307/2575191  420
result: 10.1007/s11251-006-9005-2  200
result: 10.1109/BigData.2013.6691674  200
result: 10.1109/CultureComputing.2013.61  200
result: 10.1007/978-3-642-04898-2  200
result: 10.1126/science.1199644  200
result: 10.1016/S0020-0190(02)00447-7  200
result: 10.1111/cura.12020  403
result: 10.1093/llc/fqt066  403
result: 10.1093/llc/fqv010  403
result: 10.1179/2047058413Y.0000000115  200
result: 10.1108/00220411111105470  200
result: 10.1093/llc/fqu007  403
result: 10.1080/01930826.2013.756680  200


result: 10.1093/llc/fqr031  403
result: 10.1075/ijcl.17.3.04har  200
result: 10.2200/S00334ED1V01Y201102WBE001  200
result: 10.1093/llc/fqm006  403
result: 10.2312/eurovisstar.20151113  200
result: 10.1017/S0954394510000207  200
result: 10.1007/978-1-4471-6596-5˙11  404
result: 10.1145/2835776.2835827  200
result: 10.1093/alh/ajv029  403
result: 10.3998/etlc.9362034.0001.001  200
result: 10.1007/978-3-319-24129-6_36  200
result: 10.1007/978-1-4471-4072-6  200
result: 10.1145/2133806.2133826  200
result: 10.1525/rep.2014.125.1.80  403
result: 10.1016/j.poetic.2013.08.005  200
result: 10.1016/j.poetic.2013.08.002  200
result: 10.1007/978-3-642-35834-0_24  200
result: 10.1007/978-3-540-31839-2_7  200
result: 10.1007/978-3-642-40501-3_68  200
result: 10.1007/978-3-642-33290-6_50  200
result: 10.1080/00144940.2013.875873  200
result: 10.13012/j8td9v7m  200
result: 10.1045/march2006-crane  200
result: 10.1007/978-0-585-35958-8_20  200
result: 10.1007/978-1-4471-6596-5_11  200
result: 10.1111

result: 10.1177/0956797613518349  200
result: 10.4242/BalisageVol5.Huitfeldt01  200
result: 10.1086/693968  200
result: 10.1109/JCDL.2003.1204888  200
result: 10.1093/llc/fqx024  403
result: 10.5749/vergstudglobasia.2.2.0058  420
result: 10.1109/ICDAR.2017.226  200
result: 10.1109/ICDAR.2017.59  200
result: 10.1109/DAS.2016.74  200
result: 10.1016/j.patcog.2011.07.005  200
result: 10.1045/march2006-crane  200
result: 10.1109/ICDAR.2017.225  200
result: 10.1086/694112  200
result: 10.1109/ICFHR.2014.67  200
result: 10.1007/978-3-319-70863-8_23  200
result: 10.1007/s00799-015-0138-2  200
result: 10.1007/978-3-319-70863-8_30  200
result: 10.20730/200006198  200
result: 10.1353/jjs.2015.0062  503
result: 10.11588/dah.2015.1.21631  200
result: 10.1088/1742-5468/2008/10/P10008  200
result: 10.3917/socio.081.0057  403
result: 10.1177/0539018412456918  200
result: 10.3917/ride.257.0321  403
result: 10.1145/2464464.2464475  200
result: 10.1177/1461444814567981  200
result: 10.3917/res.195.0085 

result: 10.22032/dbt.34849  200
result: 10.1145/2595188.2595193  200
result: 10.1016/j.patcog.2018.08.020  200
result: 10.1093/llc/fqw064  403
result: 10.4000/jtei.1114  200
result: 10.5281/zenodo.2543455  200
result: 10.18452/19407  200
result: 10.4000/jtei.523  200
result: 10.1145/3132218.3132237  200
result: 10.3233/ISU-170852  200
result: 10.1145/2686873  200
result: 10.5281/zenodo.45042  200
result: 10.3115/1119250.1119276  403
result: 10.30006/JC.201110.0002  200
result: 10.5539/ass.v9n5p18  200
result: 10.1126/science.aal1012  200
result: 10.6084/m9.figshare.6683855  200
result: 10.1038/nature.2017.21800  200
result: 10.1038/502295a  200
result: 10.1038/d41586-018-00104-7  200
result: 10.1038/sdata.2016.18  200
result: 10.1007/s10579-011-9157-9  200
result: 10.1007/s00357-006-0017-z  200
result: 10.1103/PhysRevE.70.025101  200
result: 10.1007/BF01908075  200
result: 10.1371/journal.pone.0018961  200
result: 10.1103/PhysRevE.76.036106  200
result: 10.1073/pnas.0611034104  200
res

result: 10.1145/1963405.1963504  403
result: 10.1073/pnas.1416284111  200
result: 10.1016/0160-7383(88)90076-X  200
result: 10.1080/01973762.2013.761106  200
result: 10.5860/crl.76.5.690  200
result: 10.5613/rzs.41.2.1  200
result: 10.1386/eta.10.1.99_1  200
result: 10.2307/3193532  420
result: 10.1080/10548408.2011.615245  200
result: 10.1080/1472586x.2012.642958  200
result: 10.5040/9781472544988  200
result: 10.1045/march2016-padilla  200
result: 10.1016/j.tourman.2013.05.007  200
result: 10.1108/lht-07-2015-0073  200
result: 10.1080/1369118X.2012.678878  200
result: 10.1145/2485760.2485874  200
result: 10.1093/llc/fqq020  403
result: 10.2307/4486062  403
result: 10.1093/llc/fqn021  403
result: 10.3366/ijhac.2014.0117  200
result: 10.1145/1921614.1921615  200
result: 10.1007/978-3-319-11964-9_27  200
result: 10.1145/996350.996432  403
result: 10.1145/2566486.2567973  200
result: 10.1093/llc/fqt037  403
result: 10.1080/03637750009376501  200
result: 10.1371/journal.pone.0094093  200


result: 10.1080/01930826.2013.756680  200
result: 10.1080/01930826.2013.756699  200
result: 10.1109/ETVIS.2016.7851166  200
result: 10.1109/TVCG.2017.2674958  200
result: 10.1145/192426.192435  403
result: 10.1145/1054972.1054990  403
result: 10.1111/cgf.12104  403
result: 10.1145/288392.288585  403
result: 10.1145/1141277.1141471  403
result: 10.1007/978-3-642-39094-4_33  200
result: 10.1109/ICDAR.2003.1227735  200
result: 10.1109/32.177365  200
result: 10.1109/ICDAR.2001.953806  200
result: 10.1145/336597.336632  403
result: 10.1145/2702613.2732778  200
result: 10.1109/TVCG.2016.2618797  200
result: 10.1145/964696.964702  403
result: 10.1016/j.patcog.2006.04.043  200
result: 10.1145/1753326.1753418  403
result: 10.1109/2945.981848  200
result: 10.1145/223904.223912  403
result: 10.1145/2207676.2208327  200
result: 10.1145/1054972.1055054  403
result: 10.1145/2396636.2396670  200
result: 10.1111/cgf.12873  403
result: 10.1109/PACIFICVIS.2015.7156366  200
result: 10.1109/TVCG.2010.194 

,DOI,status
0,10.1045/november2010-byrne,200
1,10.1080/19386389.2013.826095,200
2,10.1080/19411260903035809,200
3,10.1353/cea.2014.0014,503
4,10.6084/m9.figshare.1092550%20,200


In [225]:
df_db_st = df_http_status['status'].value_counts()
df_db_st.head(10)

200    796
403    278
404     32
503     24
420     21
408      7
500      3
406      1
Name: status, dtype: int64

### HTTP Return Status: 404 not found

There are 30 distinct references whose DOIs return 404

In [226]:
df_404=df_http_status[df_http_status['status'] == 404]
test=df_404.join(df_refs_with_doi.set_index('DOI'), on='DOI')
test.drop_duplicates(subset='DOI', keep='first').count()

DOI          30
status       30
Reference    30
dtype: int64

In [227]:
#example of DOIs returning 404
test.drop_duplicates(subset='DOI', keep='first').reset_index(drop=True).head().style.set_properties(subset=['Reference'], **{'width': '600px'})
#test.to_csv(path_or_buf='../data/notfound.csv', sep=';')

,DOI,status,Reference
0,10.1093/llc/fqn03,404,"Lavagnino, J. (2009). Access. Literary and Linguistic Computing, 24 (1) (April), 10.1093/llc/fqn03, http://llc.oxfordjournals.org/content/24/1/63.full.pdf+html."
1,10.1093/llc/gqv010,404,"Eder, M. (2015a). Rolling stylometry. Digital Scholarship in the Humanities , 30 , first published online: 7 April 2015, doi: 10.1093/llc/gqv010."
2,10.4000/socio.1338,404,"Mounier, P. (2015). Une « utopie politique » pour les humanités numériques ?. Socio , 4 : 97–112. doi:10.4000/socio.1338."
3,10.3318/DRI.2012.1,404,"O’Carroll, Aileen and Webb, Sharon : Digital Archiving in Ireland: National Survey of the Humanities and Social Sciences. Maynooth University (2012) DOI: 10.3318/DRI.2012.1"
4,10.9776/13289,404,"Kou, Y. and Nardi, B. (2013). Regulating anti-social behavior on the Internet: The example of League of Legends. iConference 2013 Proceedings , Fort Worth, TX: February 12-15, 2013, pp. 616-22. 10.9776/13289."


### Retrieve citation DOIs using CrossRef API
Crossref API allows querying the database by giving it in input strings that contain bibliography references. The reference string does not to be necessarily a well-written references. The input string is parsed by Crossref using machine learning techniques and the system tries to match the reference string with the metadata that are stored in the database. 

An important feature of Crossref API, is the score of sureness that Crossref API retrieve beside the document’s metadata. For each request, Crossref score indicates how much it is sure about the entities retrieved, if the score value is high the metadata retrieved are probably the corrected ones, if the score is low the metadata retrieved might be the wrong ones.

Here the Crossref query API is used to check if citations whose DOI http connection returns "404 not found" can be found in the Crossref Database and, if yes, what is their DOI. The value *110* is the minimum score value.

In [228]:
import urllib.request, json
references=test['Reference'];
df_citations404 = pd.DataFrame(columns=["Publication_cit", "Crossref_cit_title", "Crossref_cit_author", "Crossref_cit_metadata", "Crossref_DOI"])
originalCitations=[]
crossrefCitations=[]
crossrefCitationsAuthor=[]
crossrefCitationsMetadata=[]
DOIs=[]
score=[]
i=0;
j =0;
for citenf in references:
    cit=urllib.parse.quote_plus(citenf)
    try:
        with urllib.request.urlopen("https://api.crossref.org/works?query.bibliographic="+cit+"&sort=score&mailto=cesare.concordia@gmail.com#", timeout=18000) as url:
            data16 = json.loads(url.read().decode())
            j=j+1
            if (j%5 == 0):
                print(f"{j}, ({i})")
            if (len(data16["message"]["items"])>0) and (data16["message"]["items"][0]['score'] >110):
                originalCitations.append(citenf)
                crossrefCitationsMetadata.append( data16["message"]["items"][0])
                crossrefCitations.append( data16["message"]["items"][0]['title'])
                crossrefCitationsAuthor.append( data16["message"]["items"][0]['author'])
                DOIs.append(data16["message"]["items"][0]['DOI'])
                score.append(data16["message"]["items"][0]['score'])
                i=i+1
                #print(f"{i} found, out of {j}")
            if (j>1000):
                break
    except urllib.error.URLError:
        print(cit)
    except urllib.error.HTTPError:
        print(cit)
        
df_citations404["Publication_cit"] = originalCitations
df_citations404["Crossref_cit_title"] = crossrefCitations
df_citations404["Crossref_cit_author"] = crossrefCitationsAuthor
df_citations404["Crossref_cit_metadata"] = crossrefCitationsMetadata
df_citations404["Crossref_DOI"] = DOIs
df_citations404["Score"] = score
df_citations404.head()

5, (0)
10, (0)
15, (0)
20, (2)
25, (4)
30, (6)
35, (8)


,Publication_cit,Crossref_cit_title,Crossref_cit_author,Crossref_cit_metadata,Crossref_DOI,Score
0,"Griffiths Mary and Kim Barbour. ""‘Imagine If O...",['Imagine if our cities talked to us': Questio...,"[{'given': 'Mary', 'family': 'Griffiths', 'seq...","{'indexed': {'date-parts': [[2020, 3, 28]], 'd...",10.20851/publics-03,174.881270
1,"Underwood, T., Bamman, D. and Lee, S. (2018). ...",[The Transformation of Gender in English-Langu...,"[{'given': 'Ted', 'family': 'Underwood', 'sequ...","{'indexed': {'date-parts': [[2020, 10, 23]], '...",10.22148/16.019,116.795525
2,"Bates, M. (1998). Indexing and access for digi...",[Indexing and access for digital libraries and...,"[{'given': 'Marcia J.', 'family': 'Bates', 'se...","{'indexed': {'date-parts': [[2020, 12, 5]], 'd...",10.1002/(sici)1097-4571(1998110)49:13<1185::ai...,111.930290
3,"Spiro, L. (2012). “This Is Why We Fight”: Defi...",[“This Is Why We Fight”: Defining the Values o...,"[{'given': 'Lisa', 'family': 'Spiro', 'sequenc...","{'indexed': {'date-parts': [[2020, 3, 2]], 'da...",10.5749/minnesota/9780816677948.003.0003,136.291180
4,"Shapiro, L. A. (2012). Embodied Cognition. In ...",[6. Embodied Cognition],"[{'given': 'Lawrence A.', 'family': 'Shapiro',...","{'indexed': {'date-parts': [[2020, 12, 6]], 'd...",10.1093/oxfordhb/9780195309799.003.0006,114.842720


Using the Crossref API, 10 DOIs have been found

In [237]:
df_citations404.count()
#df_citations404[['Publication_cit', 'Crossref_cit_title', 'Crossref_cit_author', 'Crossref_DOI']].head(10).style.set_properties(subset=['Publication_cit'], **{'width': '200px'})

Publication_cit          10
Crossref_cit_title       10
Crossref_cit_author      10
Crossref_cit_metadata    10
Crossref_DOI             10
Score                    10
dtype: int64

In [236]:
retrieved404=df_citations404.join(test.set_index('Reference'), on='Publication_cit',  lsuffix='_in_publication')
retrieved404[['Publication_cit', 'Crossref_cit_title', 'Crossref_cit_author','Crossref_DOI', 'DOI']].head(10).style.set_properties(subset=['Publication_cit','Crossref_cit_title'], **{'width': '200px'})

,Publication_cit,Crossref_cit_title,Crossref_cit_author,Crossref_DOI,DOI
0,"Griffiths Mary and Kim Barbour. ""‘Imagine If Our Cities Talked to Us’: Questions about the Making of ‘responsive’ Places and Urban Publics."" In Making Publics, Making Places, 27-48. South Australia: University of Adelaide Press, 2016, http://www.jstor.org/stable/10.20851/j.ctt1t304qd.8","[""'Imagine if our cities talked to us': Questions about the making of 'responsive' places and urban publics""]","[{'given': 'Mary', 'family': 'Griffiths', 'sequence': 'first', 'affiliation': []}]",10.20851/publics-03,10.20851/j.ctt1t304qd.8
1,"Underwood, T., Bamman, D. and Lee, S. (2018). The transformation of Gender in English language fiction. Cultural Analytics, http://culturalanalytics.org/2018/02/thetransformation-of-gender-in-english-language-fiction/ (DOI: 10.7910/DVN/TEGMGI)",['The Transformation of Gender in English-Language Fiction'],"[{'given': 'Ted', 'family': 'Underwood', 'sequence': 'first', 'affiliation': []}, {'given': 'David', 'family': 'Bamman', 'sequence': 'additional', 'affiliation': []}, {'given': 'Sabrina', 'family': 'Lee', 'sequence': 'additional', 'affiliation': []}]",10.22148/16.019,10.7910/DVN/TEGMGI
2,"Bates, M. (1998). Indexing and access for digital libraries and the Internet: human, database, and domain factors. Journal of the American Society for Information Science , 49 (13) doi:10.1002/(SICI)1097-4571(1998110)49:13<1185::AID-ASI6>3.3.CO;2-M (accessed 23 March 2011).","['Indexing and access for digital libraries and the internet: Human, database, and domain factors']","[{'given': 'Marcia J.', 'family': 'Bates', 'sequence': 'first', 'affiliation': []}]",10.1002/(sici)1097-4571(1998110)49:13<1185::aid-asi6>3.0.co;2-v,10.1002/(SICI)1097-4571(1998110)49:13
3,"Spiro, L. (2012). “This Is Why We Fight”: Defining the Values of the Digital Humanities. In M. K. Gold (ed.), Debates in the Digital Humanities . University of Minnesota Press, pp. 16–35. Available at http://minnesota.universitypressscholarship.com/view/10.5749/minnesota/9780816677948.001.0001/upso-9780816677948-chapter-3",['“This Is Why We Fight”: Defining the Values of the Digital Humanities'],"[{'given': 'Lisa', 'family': 'Spiro', 'sequence': 'first', 'affiliation': []}]",10.5749/minnesota/9780816677948.003.0003,10.5749/minnesota/9780816677948.001.0001/upso-9780816677948-chapter-3
4,"Shapiro, L. A. (2012). Embodied Cognition. In Margolis, E., Samuels, R., and Stich, S. P. (eds.), The Oxford Handbook of Philosophy of Cognitive Science . Oxford, New York: Oxford University Press, pp. 118–46. http://www.oxfordhandbooks.com/view/10.1093/oxfordhb/9780195309799.001.0001/oxfordhb-9780195309799-e-6 (accessed 26 November 2018).",['6. Embodied Cognition'],"[{'given': 'Lawrence A.', 'family': 'Shapiro', 'sequence': 'first', 'affiliation': []}]",10.1093/oxfordhb/9780195309799.003.0006,10.1093/oxfordhb/9780195309799.001.0001/oxfordhb-9780195309799-e-6
5,"Hirsch, Brett D., 2011. The Kingdom Has Been Digitized: Electronic Editions of Renaissance Drama and the Long Shadows of Shakespeare and Print . Literature Compass [online] 8 (9): 568-591. Available from: http://onlinelibrary.wiley.com/doi/10.1111/lico.2011.8.issue-9/issuetoc [Accessed 27 September 2012].",['The Kingdom has been Digitized: Electronic Editions of Renaissance Drama and the Long Shadows of Shakespeare and Print'],"[{'given': 'Brett D.', 'family': 'Hirsch', 'sequence': 'first', 'affiliation': []}]",10.1111/j.1741-4113.2011.00830.x,10.1111/lico.2011.8.issue-9/issuetoc
6,"Romanello, Matteo. 2013. Creating an Annotated Corpus for Extracting Canonical Citations from Classics-Related Texts by Using Active Annotation. In Computational Linguistics and Intelligent Text Processing. 14th International Conference, CICLing 2013, Samos, Greece, March 24-30, 2013, Proceedings, Part I , edited by Alexander Gelbukh, 1:60–76. Lecture Notes in Computer Science / Theoretical Computer Science and General Issues. Springer Berlin Heidelberg. doi:10.100

In [233]:
retrieved404.Crossref_DOI.values

array(['10.20851/publics-03', '10.22148/16.019',
       '10.1002/(sici)1097-4571(1998110)49:13<1185::aid-asi6>3.0.co;2-v',
       '10.5749/minnesota/9780816677948.003.0003',
       '10.1093/oxfordhb/9780195309799.003.0006',
       '10.1111/j.1741-4113.2011.00830.x', '10.1007/978-3-642-37247-6_6',
       '10.1002/9781118680605.ch23', '10.1002/9781405177504.ch28',
       '10.1207/s15327051hci1304_2'], dtype=object)

In [231]:
dfdc=get_HTTP_Status(retrieved404.Crossref_DOI.values)

result: 10.20851/publics-03  200
result: 10.22148/16.019  200
result: 10.1002/(sici)1097-4571(1998110)49:13<1185::aid-asi6>3.0.co;2-v  403
result: 10.1111/j.1741-4113.2011.00830.x  403
result: 10.1007/978-3-642-37247-6_6  200
result: 10.1002/9781118680605.ch23  403
result: 10.1002/9781405177504.ch28  403
result: 10.1207/s15327051hci1304_2  200


In [232]:
dfdc_st = dfdc['status'].value_counts()
dfdc_st.head()

403    4
200    4
408    2
Name: status, dtype: int64

8 retrieved DOIs are valid (HTTP 403 is a HTTP status code meaning that he server understood the request, but will not fulfill it due to client-related issues), 2 of them returns connection timeout status.

In [247]:
dfdc.head(10)

,DOI,status
0,10.20851/publics-03,200
1,10.22148/16.019,200
2,10.1002/(sici)1097-4571(1998110)49:13<1185::ai...,403
3,10.5749/minnesota/9780816677948.003.0003,408
4,10.1093/oxfordhb/9780195309799.003.0006,408
5,10.1111/j.1741-4113.2011.00830.x,403
6,10.1007/978-3-642-37247-6_6,200
7,10.1002/9781118680605.ch23,403
8,10.1002/9781405177504.ch28,403
9,10.1207/s15327051hci1304_2,200


In [253]:
dfdc_t=dfdc.join(retrieved404[['Publication_cit', 'Crossref_DOI']].set_index('Crossref_DOI'), on='DOI',  lsuffix='_')
dfdc_t.head(10).style.set_properties(subset=['Publication_cit'], **{'width': '300px'})

,DOI,status,Publication_cit
0,10.20851/publics-03,200,"Griffiths Mary and Kim Barbour. ""‘Imagine If Our Cities Talked to Us’: Questions about the Making of ‘responsive’ Places and Urban Publics."" In Making Publics, Making Places, 27-48. South Australia: University of Adelaide Press, 2016, http://www.jstor.org/stable/10.20851/j.ctt1t304qd.8"
1,10.22148/16.019,200,"Underwood, T., Bamman, D. and Lee, S. (2018). The transformation of Gender in English language fiction. Cultural Analytics, http://culturalanalytics.org/2018/02/thetransformation-of-gender-in-english-language-fiction/ (DOI: 10.7910/DVN/TEGMGI)"
2,10.1002/(sici)1097-4571(1998110)49:13<1185::aid-asi6>3.0.co;2-v,403,"Bates, M. (1998). Indexing and access for digital libraries and the Internet: human, database, and domain factors. Journal of the American Society for Information Science , 49 (13) doi:10.1002/(SICI)1097-4571(1998110)49:13<1185::AID-ASI6>3.3.CO;2-M (accessed 23 March 2011)."
3,10.5749/minnesota/9780816677948.003.0003,408,"Spiro, L. (2012). “This Is Why We Fight”: Defining the Values of the Digital Humanities. In M. K. Gold (ed.), Debates in the Digital Humanities . University of Minnesota Press, pp. 16–35. Available at http://minnesota.universitypressscholarship.com/view/10.5749/minnesota/9780816677948.001.0001/upso-9780816677948-chapter-3"
4,10.1093/oxfordhb/9780195309799.003.0006,408,"Shapiro, L. A. (2012). Embodied Cognition. In Margolis, E., Samuels, R., and Stich, S. P. (eds.), The Oxford Handbook of Philosophy of Cognitive Science . Oxford, New York: Oxford University Press, pp. 118–46. http://www.oxfordhandbooks.com/view/10.1093/oxfordhb/9780195309799.001.0001/oxfordhb-9780195309799-e-6 (accessed 26 November 2018)."
5,10.1111/j.1741-4113.2011.00830.x,403,"Hirsch, Brett D., 2011. The Kingdom Has Been Digitized: Electronic Editions of Renaissance Drama and the Long Shadows of Shakespeare and Print . Literature Compass [online] 8 (9): 568-591. Available from: http://onlinelibrary.wiley.com/doi/10.1111/lico.2011.8.issue-9/issuetoc [Accessed 27 September 2012]."
6,10.1007/978-3-642-37247-6_6,200,"Romanello, Matteo. 2013. Creating an Annotated Corpus for Extracting Canonical Citations from Classics-Related Texts by Using Active Annotation. In Computational Linguistics and Intelligent Text Processing. 14th International Conference, CICLing 2013, Samos, Greece, March 24-30, 2013, Proceedings, Part I , edited by Alexander Gelbukh, 1:60–76. Lecture Notes in Computer Science / Theoretical Computer Science and General Issues. Springer Berlin Heidelberg. doi:10.1007/978-3-642-37247-6\_6 ."
7,10.1002/9781118680605.ch23,403,"Zundert, van, J.J., 2016. Screwmeneutics and Hermenumericals: the Computationality of Hermeneutics. In S. Schreibman, R. Siemens, & J. Unsworth, eds. A New Companion to Digital Humanities . Malden (US), Oxford (UK), etc.: John Wiley & Sons, Ltd, pp. 331–347. Available at: http://onlinelibrary.wiley.com/doi/10.1002/9781118680605.ch23/summary [Accessed July 26, 2016]."
8,10.1002/9781405177504.ch28,403,"Hoover, D. L. (2013). Quantitative Analysis and Literary Studies. In R. Siemens & S. Schreibman (Eds.), A Companion to Digital Literary Studies (pp. 517–533). John Wiley & Sons, Ltd. Retrieved from http://onlinelibrary.wiley.com/doi/10.1002/9781405177504.ch28/summary"
9,10.1207/s15327051hci1304_2,200,"Dourish, Paul, and Graham Button. 1998. On ‘Technomethodology’: Foundational Relationships between Ethnomethodology and System Design. Human-Computer Interaction 13 (4): 395–432. https://doi.org/10.1207/s15327051hci1304 ."
